# City Quality of Life 
### DOPP Group 9

In [ ]:
import pandas as pd
import eurostat
import numpy as np
import os

pd.set_option('display.max_rows', 50)
pd.options.mode.chained_assignment = None


## Response: Quality of Life ranking from Numbeo 

In [ ]:
from Response_Variable import ReponseVariableScraper

european_countries=["Switzerland","Netherlands","Denmark",
"Austria","Luxembourg","Iceland","United Kingdom","Germany",
"Spain","Estonia","Sweden","Ireland","Slovenia","Lithuania",
        "Turkey","Czech Republic","Norway","Croatia",
        "France","Belgium","Portugal","Cyprus","Romania","Poland","Slovakia",
        "Latvia","Russia","Italy","Bulgaria","Serbia","Greece",
"Hungary","Ukraine"]

def filter_europe(df):
    df=df[df["Country"].isin(european_countries)]
    return df


data_scraper = ReponseVariableScraper()

#### Ranking & Score Aggregation
After the necessary preprocessing has been done the next step in our time analysis is to summerize all the dataset into a dictionary containing the response variable, the ranking and city for every year.
Furthermore, two a dataframe was created which counts for every city how often it scored in the top 10 by ranking during the time period 2012-2020. Additionally, a dataframe which accumulates for every
city the scores that it received over the years. At last both dataframes were sorted by their counts of ranking or accumulated scores in descending order.



In [ ]:
data={}
for year in range(2012,2021):
    data[year]=data_scraper.get_year(year)
    data[year]=filter_europe(data[year])
    data[year]["Rank"]=range(1,len(data[year])+1)
all_cities=[]
for year in range(2012,2021):
    all_cities.extend(data[year]["City"].unique())

all_cities=np.unique(all_cities)
cities=pd.DataFrame(columns=["City","Top_10"])
cities["City"]=all_cities
cities["Top_10"]=0
QOL_acc=pd.DataFrame(columns=["City","QOL_acc"])
QOL_acc["City"]=all_cities
QOL_acc["QOL_acc"]=0
for year in range(2012,2021):
    df=data[year]
    df.sort_values(by="City",inplace=True)
    candidates=df[df["Rank"]<=10]["City"].to_list()
    idx_c=cities["City"].isin(candidates)
    cities["Top_10"].loc[idx_c]+=1
    c=df["City"].to_list()
    idx_acc=QOL_acc["City"].isin(c)
    q=df["Quality of Life Index"].to_list()
    QOL_acc["QOL_acc"][idx_acc]+=q
cities.sort_values(by=["Top_10"],ascending=False,inplace=True)

qol_response = data_scraper.get_interpolated_years()


## reshape (melt) into city-year : value shape, convert year to int to match others
response = qol_response.melt(id_vars=['City'], value_vars=['2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012'],\
                         var_name = 'Year', value_name='QOL')
response['Year'] = response['Year'].astype(int)
response.head(1)

## Pollution

In [ ]:
!python Pollution_Scraper.py

In [ ]:
## for each year: read QOL + pollution csv, merge, add year, append to combined df 
pollution = pd.DataFrame()    
for year in range(2012,2021):
    ## read pollution for year
    filename = ('../data/' + 'Pollution_' + str(year))
    df_yr = pd.read_csv(filename).iloc[:,2:4]
    
    ## read Quality of Life for year
    #filename_qol = ('../../data/' + 'Quality_of_life_' + str(year))
    #df_qol = pd.read_csv(filename_qol).iloc[:,2:4]
    
    # merging QOL & Pollution 
    #df_yr = pd.merge(df_yr,qol_y, how = 'outer', on = 'City')
    
    ## define year and rename
    df_yr.loc[:,'Year'] = int(year)
    df_yr = df_yr.rename(columns = {'Pollution Index':'Pollution','Quality of Life Index':'QOL'})
    
    ## concat to combined df
    pollution = pd.concat([pollution, df_yr])

#df_all.to_csv('../data/Pol_merged.csv', index = False)
pollution.head(1)

### Climate: Cooling and heating degree days by NUTS 2 regions

In [ ]:
## get eurostat data
heatdays = eurostat.get_data_df('nrg_chddr2_a', flags=False)
heatdays = heatdays.iloc[:,:11].rename(columns={'geo\\time': ' NUTS 2'})

## Merge on all entries which are also in the target variable cities to extract only the interesting cities
target_cities = pd.read_csv("../data/Cities_with_codes.csv")
heatdays_nuts = pd.merge(target_cities, heatdays, on=[' NUTS 2'])

## split into cool & heating days
heatdays_cdd = heatdays_nuts.iloc[np.where(heatdays_nuts.indic_nrg == 'CDD')]
heatdays_hdd = heatdays_nuts.iloc[np.where(heatdays_nuts.indic_nrg == 'HDD')]

## reshape (melt) into city-year : value shape
heat = heatdays_hdd.melt(id_vars=['City'], value_vars=[2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012],\
                         var_name = 'Year', value_name='Heatdays')
cool = heatdays_cdd.melt(id_vars=['City'], value_vars=[2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012],\
                         var_name = 'Year', value_name='Cooldays')
cool.shape
#heatdays_cdd.to_csv('../data/heatdays_hdd.csv')
#heatdays_hdd.head()

### Education Attainment


In [ ]:
# Import the data from Eurostat
df = eurostat.get_data_df('edat_lfse_04')
df.columns = df.columns.astype(str)

# Drop all years before 2012 and columns unit and age (same for all entries)
df = df.drop(df.loc[:, '2011': '2000'].columns, axis = 1)
df = df.drop(['unit', 'age'], axis = 1)

# Keep only total values and discard the gender column
df = df[df.sex == 'T']
df = df.drop('sex', axis = 1)

# Rename to avoid problems using \
df = df.rename(columns={'geo\\time': ' NUTS 2'})

# Merge on all entries which are also in the target variable cities to extract only the interesting cities
#target_cities = pd.read_csv("../data/Cities_with_codes.csv")
education_attainment = pd.merge(target_cities, df, on=[' NUTS 2'])

# Glasgow is missing all values except 2012
Glasgow = education_attainment.loc[education_attainment['City'] == 'Glasgow']
# Whole region of Scotland is UKM
# Glasgow (Metro area) is approx 33% of the population it should be a reasonable approximation
temp  = df[df[' NUTS 2'] == 'UKM']
temp = temp.loc[:,'2019':'2013']
Glasgow.loc[:, '2019':'2013'] = temp.loc[:, '2019':'2013'].to_numpy()
education_attainment[education_attainment['City'] == 'Glasgow'] = Glasgow


# Impute the missing values from 2012 using padding
education_attainment.loc[:, '2019':'2012'] = education_attainment.loc[:, '2019':'2012'].fillna(method='pad',axis=1)

# Split in to 3 separate tables depending on education level
edu_ED_0_2 = education_attainment[education_attainment['isced11'] == 'ED0-2']
edu_ED_3_4 = education_attainment[education_attainment['isced11'] == 'ED3_4']
edu_ED_5_8 = education_attainment[education_attainment['isced11'] == 'ED5-8']

#To be able to merge with the other data
#Transform the columns of each year to a variable year
yearly_data = dict()
education_attainment  = pd.DataFrame()
for year in range(2012,2020):
    yearly_data= edu_ED_0_2[ list(edu_ED_0_2.loc[:,'City':' Country']) + [f"{year}"]]
    yearly_data.insert(4, "Year", year)
    yearly_data = yearly_data.rename(columns={f"{year}": "Education Attainment ED 0-2"})
    yearly_data.insert(6, "Education Attainment ED 3-4", edu_ED_3_4[f"{year}"].to_numpy())
    yearly_data.insert(7, "Education Attainment ED 5-8", edu_ED_5_8[f"{year}"].to_numpy())
    education_attainment = education_attainment.append(yearly_data)
    
education_attainment = education_attainment.reset_index(drop=True)

### Victims in Road Accidents

In [ ]:
# Get the data from Eurostat
df = eurostat.get_data_df('tran_r_acci')
df.columns = df.columns.astype(str)
# Drop all years before 2012, keep only with unit measure Per Million Inhabitants and accident type deadly
df = df.drop(df.loc[:, '2011': ].columns, axis = 1)
df = df[df.unit == 'P_MHAB']
df = df[df.victim == 'KIL']

# Rename to avoid problems using \
df = df.rename(columns={'geo\\time': ' NUTS 2'})

# Insert missing cloumn for 2019
df.insert(3,"2019",np.NaN)

# Merge on all entries which are also in the target variable cities to extract only the interesting cities
road_accidents = pd.merge(target_cities, df, on=[' NUTS 2'])

# Check for missing cities
missing_cities = target_cities[-target_cities[' NUTS 2'].isin(road_accidents[' NUTS 2'])]


print("Missing cities: ", missing_cities.values[:,0])

# Data from Icelandic ministry of transportation
#https://www.samgongustofa.is/umferd/tolfraedi/slysatolur/arsskyrslur-slysaskraningar/
# Reported as deaths per 10,000
Reykjavik  = [0.0, 30.0, 20.0, 10.0, 20.0, 0.0, 10.0, 10.0]
Reykjavik = ['Reykjavik', 'IS001C1','IS00','IS','KIL','P_MHAB']+Reykjavik
road_accidents.loc[77] = Reykjavik

# Table 1-1 in : https://www.abs.gov.rs/admin/upload/documents/20181016102533-statistical_report_2016_english.pdf
# 20% of the population is in Belgrade so we take 20% of the accidents as occuring there
# and divide by 1.7 (million inhabitants)
Belgrade = [np.NaN, np.NaN,np.NaN, 619, 594, 476, 548, 551]
Belgrade = np.multiply(Belgrade,(0.2/1.7))
Belgrade = np.around(Belgrade, 1)
Belgrade = ['Belgrade', '-','RS11','RS','KIL','P_MHAB']+Belgrade.tolist()
road_accidents.loc[78] = Belgrade

# Traffic death data from Scotland
# https://statistics.gov.scot/data/road-safety
# Select by region

# Divide by 0.5 (million inhabitants)
Edinburgh = [np.NaN, 5, 6, 9, 3, 11, 8, 13]
Edinburgh = np.divide(Edinburgh,0.5)
Edinburgh = np.around(Edinburgh, 1)
Edinburgh = ['Edinburgh', 'UK007C1','UKM7','UK','KIL','P_MHAB']+Edinburgh.tolist()
road_accidents.loc[79] = Edinburgh

# Divide by 0.6 (million inhabitants)
Glasgow = [np.NaN, 10, 7, 8, 15, 18, 4, 7]
Glasgow = np.divide(Glasgow,0.6)
Glasgow = np.around(Glasgow, 1)
Glasgow = ['Glasgow', 'UK004C1','UKM3','UK','KIL','P_MHAB']+Glasgow.tolist()
road_accidents.loc[80] = Glasgow

# Impute using padding to fill missing values
road_accidents.loc[:, '2019':'2013'] = road_accidents.loc[:, '2019':'2012'].fillna(method='backfill',axis=1)
road_accidents.loc[:, '2019':'2012'] = road_accidents.loc[:, '2019':'2012'].fillna(method='ffill',axis=1)

#To be able to merge with the other data
#Transform the columns of each year to a variable year
yearly_data = dict()
deaths_in_road_accidents  = pd.DataFrame()
for year in range(2012,2020):
    yearly_data= road_accidents[ list(road_accidents.loc[:,'City':' Country']) + [f"{year}"]]
    yearly_data.insert(4, "Year", year)
    yearly_data = yearly_data.rename(columns={f"{year}": "Deaths_in_road_accidents"})
    deaths_in_road_accidents = deaths_in_road_accidents.append(yearly_data)
deaths_in_road_accidents = deaths_in_road_accidents.reset_index(drop=True)

### Average Working Hours

In [ ]:
# Load the datasheet from eurostat
df = eurostat.get_data_df('lfst_r_lfe2ehour')

# Drop all years before 2012 and columns unit and age (same for all entries)
df.columns = df.columns.astype(str)
df = df.drop(df.loc[:, '2011': ].columns, axis = 1)
# Rename to avoid problems using \
df = df.rename(columns={'geo\\time': ' NUTS 2'})

# Extract the data for both sexes (Total) and age group 15-74
df = df[df.sex == 'T']
df = df.drop('sex', axis = 1)
df = df[df.age == 'Y15-74']

# Merge on all entries which are also in the target variable cities to extract only the interesting cities
avg_hours = pd.merge(target_cities, df, on=[' NUTS 2'])

# Glasgow is missing all values except 2012
Glasgow = avg_hours.loc[avg_hours['City'] == 'Glasgow']
# Whole region of Scotland is UKM
# Glasgow is 40% of the population it should be a reasonable approximation
temp  = df[df[' NUTS 2'] == 'UKM']
temp = temp.loc[:,'2019':'2013']
Glasgow.loc[:, '2019':'2013'] = temp.loc[:, '2019':'2013'].to_numpy()
avg_hours[avg_hours['City'] == 'Glasgow'] = Glasgow

# Impute the rest using padding
#avg_hours = avg_hours.interpolate(method='pad',axis=1)
avg_hours.loc[:, '2019':'2012'] = avg_hours.loc[:, '2019':'2012'].fillna(method='pad',axis=1)

#To be able to merge with the other data
#Transform the columns of each year to a variable year
yearly_data = dict()
avg_working_hours  = pd.DataFrame()
for year in range(2012,2020):
    yearly_data= avg_hours[ list(avg_hours.loc[:,'City':' Country']) + [f"{year}"]]
    yearly_data.insert(4, "Year", year)
    yearly_data = yearly_data.rename(columns={f"{year}": "Average Working Hours"})
    avg_working_hours = avg_working_hours.append(yearly_data)
avg_working_hours = avg_working_hours.reset_index(drop=True)

### Unemployment Rate

In [ ]:
# Read the datasheet from Eurostat
df = eurostat.get_data_df('lfst_r_lfu3rt')

# Drop all years before 2012 and columns unit and age (same for all entries)
df.columns = df.columns.astype(str)
df = df.drop(df.loc[:, '2011': ].columns, axis = 1)
# Rename to avoid problems using \
df = df.rename(columns={'geo\\time': ' NUTS 2'})

# Extract the data for both sexes (Total) and age group 15-74
df = df[df.sex == 'T']
df = df[df.age == 'Y15-74']
df = df.drop(['sex','unit','age'], axis = 1)

# Merge on all entries which are also in the target variable cities to extract only the interesting cities
unemployment_rate = pd.merge(target_cities, df, on=[' NUTS 2'])

# Glasgow is missing all values except 2012
Glasgow = unemployment_rate.loc[unemployment_rate['City'] == 'Glasgow']
# Whole region of Scotland is UKM
# Glasgow is 40% of the population it should be a reasonable approximation
temp  = df[df[' NUTS 2'] == 'UKM']
temp = temp.loc[:,'2019':'2013']
Glasgow.loc[:, '2019':'2013'] = temp.loc[:, '2019':'2013'].to_numpy()
unemployment_rate[unemployment_rate['City'] == 'Glasgow'] = Glasgow

# Impute the rest using padding
#unemployment_rate = unemployment_rate.interpolate(method='pad',axis=1)
unemployment_rate.loc[:, '2019':'2012'] = unemployment_rate.loc[:, '2019':'2012'].fillna(method='pad',axis=1)

#To be able to merge with the other data
#Transform the columns of each year to a variable year
yearly_data = dict()
unemployment_rate_data  = pd.DataFrame()
for year in range(2012,2020):
    yearly_data= unemployment_rate[ list(unemployment_rate.loc[:,'City':' Country']) + [f"{year}"]]
    yearly_data.insert(4, "Year", year)
    yearly_data = yearly_data.rename(columns={f"{year}": "Unemployment_Rate"})
    unemployment_rate_data = unemployment_rate_data.append(yearly_data)
unemployment_rate_data = unemployment_rate_data.reset_index(drop=True)

### Merge Education, Accidents & Working Hours

In [ ]:
# Code if we want to do it with csv files, requires all in the same format and not other existing csv files in the folder

#initial = pd.read_csv('Education_Attainment.csv')
#merged = initial
#
#for file_name in glob.glob('*.csv'):
#    if file_name == 'Education_Attainment.csv':
#        continue 
#    temp_dataframe = pd.read_csv(file_name)
#    values = temp_dataframe.drop(columns=[' City Code', ' NUTS 2',' Country'])
#    merged = merged.merge(values, on=['City', 'Year'])


# Code to do it directly in the notebook without saving to csv files first
# Right now requires all data to be complete
data_sheets = [deaths_in_road_accidents,avg_working_hours,unemployment_rate_data]

initial = education_attainment
merged = initial

for file_name in data_sheets:
    temp_dataframe = file_name
    values = temp_dataframe.drop(columns=[' City Code', ' NUTS 2',' Country'])
    merged = merged.merge(values, on=['City', 'Year'])
    
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
    
print(merged.shape)
merged.head()

## Merge all datasets

In [ ]:
join = 'inner'
#join = 'outer'

heatcool = pd.merge(heat, cool, how=join, on=['City','Year'] )
print(heatcool.shape)

heatcoolpol = pd.merge(heatcool, pollution, how=join, on=['City','Year'] )
print(heatcoolpol.shape)

heatcoolpolres = pd.merge(heatcoolpol, response , how=join, on=['City','Year'] )
print(heatcoolpolres.shape)
heatcoolpolres.head(1)

In [ ]:
alldata = pd.merge(heatcoolpolres, merged , how=join, on=['City','Year'] )
alldata = alldata.drop(columns=[' City Code', ' NUTS 2',' Country'])
print(alldata.shape)
alldata.head(3)

### Save as csv

In [ ]:
alldata.to_csv('../data/Full_data_merged.csv', index = False)

In [ ]:

# using join = 'outer' to merge
# alldata.to_csv('../data/Full_data_outer.csv', index = False)

In [ ]:
all_inner = pd.read_csv('../data/Full_data_merged.csv')
all_inner.shape